<a href="https://colab.research.google.com/github/freddyduitama/Analytics_Vidhya/blob/master/0_1_wordcount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Adecuacion de la plataforma**

In [0]:
# instala el ambiente de spark..solo se corre una vez
# en este directorio se consulta la versión activa de spark  http://apache.osuosl.org/spark/
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
!tar xf spark-2.4.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
#Configura variables de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.7"

In [0]:
#importa pyspark package
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [0]:
# define  la sesion SPARK. 
conf = SparkConf().setAppName("ejemplo").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [0]:
#Crea la sesión
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [6]:
# monta el google drive para usar sus archivos
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [7]:
!ls -l /gdrive/'My Drive'/'Colab Notebooks'

total 155
-rw------- 1 root root 30760 Jul 29 19:50 0.1.wordcount.ipynb
-rw------- 1 root root 48671 Jul 29 19:07 0.2.Introduction-To-Spark.ipynb
-rw------- 1 root root 15250 Jul 16 23:15 0.3.dataframes-1.ipynb
-rw------- 1 root root 12084 Apr 10 21:39 0.4.dataframes-2.ipynb
-rw------- 1 root root 12031 Apr  9 03:01 0.6.LSH.ipynb
-rw------- 1 root root  6342 May  7 03:57 0.7.PCA-SVD.ipynb
drwx------ 2 root root  4096 Apr 12 20:00 casos-ejemplo-old
drwx------ 2 root root  4096 Mar  6 20:16 Data
drwx------ 2 root root  4096 Mar 16 12:40 deep-learning
drwx------ 2 root root  4096 Apr 10 14:10 fastai-v3
drwx------ 2 root root  4096 Apr 10 20:19 GRD
-rw------- 1 root root   550 Mar 18 15:29 MatricesConf.cfg
-rw------- 1 root root   542 Mar 18 15:29 MatricesConf-old.cfg
-rw------- 1 root root  4725 Mar 12 22:33 Multiplicacion-matrices.ipynb
drwx------ 2 root root  4096 Jul 24 00:57 re-admision


In [0]:
#  INST A: Opcional..si quiere subir archivos al ambiente de trabajo desde su PC
from google.colab import files
datafile = files.upload()

# **Example 1.  Versión básica de longitud**

In [0]:
# OPCION 2usa la libreria basica de SPARk que lee cada entrada del RDD como una cadena de caracteres.
lines1 = sc.textFile("/gdrive/My Drive/Colab Notebooks/Data/datos.txt")

In [0]:
lines1.take(2)

['diccionario mio', 'letras tuyas']

In [0]:
lineLengths = lines1.map( lambda s : len(s) ) 

In [0]:
totalLength = lineLengths.reduce( lambda a, b : a + b)

In [0]:
print(totalLength); 

87


In [0]:
lineLengths.take(5)

[15, 12, 27, 12, 21]

# **EXAMPLE 2: Versión del word count eliminando signos de puntuación y otros caracteres**

In [0]:
#  INST A: para subir el archivo a usar en el word count
from google.colab import files
datafile = files.upload()

In [0]:
import re, string
text_file = spark.read.text("/gdrive/My Drive/Colab Notebooks/Data/texto-con-caracteres.txt").rdd
punc = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

In [0]:
# define función a usar en el map
def uni_to_clean_str(x):
    lowercased_str = x.lower()
    lowercased_str = lowercased_str.replace('--',' ')
    clean_str = lowercased_str.translate(punc)
    return clean_str

In [0]:
one_RDD = text_file.flatMap(lambda x: uni_to_clean_str(x.value).split())
one_RDD = one_RDD.map(lambda x: (x,1))
one_RDD = one_RDD.reduceByKey(lambda x,y: x + y)

In [27]:
one_RDD.take(5)

[('pride', 28), ('and', 3370), ('prejudice', 5), ('by', 624), ('jane', 156)]

In [0]:
lines=spark.read.text('/gdrive/My Drive/Colab Notebooks/Data/texto-con-caracteres.txt').rdd

In [29]:
# muestra por qué en el map de debe usar s.value, la estructura de cada entrada del RDD es row(value='xxx')
print(lines.take(2))

[Row(value='PRIDE AND PREJUDICE'), Row(value='')]


# **EXAMPLE 3: Versión básica del word count*

In [0]:
words = sc.textFile('/gdrive/My Drive/Colab Notebooks/Data/datos.txt')


In [23]:
words.take(12)

['diccionario',
 'mio',
 'letras',
 'tuyas',
 'palabras',
 'completas',
 'otra',
 'vez',
 'Prueba',
 'larga',
 'Quinta',
 'vez']

In [0]:
words1= words.flatMap(lambda line: line.split(" ")) \
                         .map(lambda word: (word, 1)) \
                         .reduceByKey(lambda a, b: a + b)

In [20]:
words1.take(5)

[('mio', 1), ('letras', 1), ('completas', 1), ('vez', 2), ('Prueba', 1)]